# Get Nestle Product Images

1. Extract product names from https://github.com/CharlesStover/peoplecott/blob/master/src/constants/children/children.ts

2. Google image search [product]+' nestle food'

### TODO:

3. Choose a random image from the first page of results

4. Tweet: "Hello friends! Don't forget, [product] is a Nestle product! Nestle is an evil company. Don't buy it!

In [271]:
from get_gis import google_image_search
from imutils.object_detection import non_max_suppression
from detect_text import detect_text
from get_twitter_api import get_twitter_api

import cv2
import glob
import os
from os import environ
import random
import re
import requests
import sys
import time
import tweepy

In [308]:
# get the freshest data from Charles Stover's github repo (peoplecott)
url='https://raw.githubusercontent.com/CharlesStover/peoplecott/master/src/constants/children/children.ts'
brands_raw_file = requests.get(url).text

In [ ]:
# idk how .ts files work but we can use regex to extract the brand names from it

#get horrible nasty raw list of all rows in the file that match any of the patterns
re1 = re.compile(r'([A-Z]+)+:', re.IGNORECASE) #text before a semicolon
re2 = re.compile(r'"([^"]*)"', re.IGNORECASE)  #text between double quotes
re3 = re.compile(r"'([^']*)'", re.IGNORECASE)  #text between single quotes
raw_list = re.findall(re1, brands_raw_file)+re.findall(re2, brands_raw_file)+re.findall(re3, brands_raw_file)

#clean up the riff-raff
exclude_list = ["http","nestle","\"",",",".","parent","source","child"]
clean_list=[]
for mystr in raw_list:
    if all(x.upper() not in mystr.upper() for x in exclude_list):
        clean_list.append(mystr)

In [248]:
#pass all these product names into a bunch of google image search URLs

#NOTE: ended up going a different way with this, but I'm keeping the code here for future reference

#url_base = 'https://www.google.com/search?q=QUERY&source=lnms&tbm=isch'
#image_search_queries = [x.replace(' ','+').replace('-','+').replace("'",'')+'+nestle+food' for x in clean_list]
#urls = [url_base.replace('QUERY',x) for x in image_search_queries]

In [289]:
clean_list[:10]

['Aero',
 'Alpo',
 'Arrowhead',
 'Beba',
 'Benecalorie',
 'Benefiber',
 'Beneful',
 'Beneprotein',
 'Bonka',
 'Boost']

In [302]:
# Choose a random item from the list
index_to_use = random.randint(0,len(clean_list))
item = clean_list[index_to_use]

# Retrieve N google image results of that item
num_images = 5
print(f"getting {num_images} images of {item}")
google_image_search(item, num_images)

getting 5 images of Haagen-Dazs
creating folder images/Haagen-Dazs...


In [303]:
# Randomly select one of the remaining images
# Check to make sure the image contains text. If it does not, delete the file and try again

image_dir=f"images/{item}"
image_files = glob.glob(f"{image_dir}/*")

x=False

while x==False:
    print(f"X is {x}, continuing...")
    image_file_index_to_use=random.randint(0,len(image_files))
    tweet_image_path=image_files[image_file_index_to_use]
    print(f"Selected image {tweet_image_path} - checking to make sure it includes text...")
    if detect_text(tweet_image_path)==True:
        print(f"Image contains text, good for the tweetening.")
        x=True
    else:   
        os.remove(tweet_image_path)
        

X is False, continuing...
Selected image images/Haagen-Dazs/Nestle-invests-in-Loop-launching-Haeagen-Dazs-in-a-reusable-container_wrbm_large.jpg - checking to make sure it includes text...
Image contains text, good for the tweetening.


In [305]:
#Tweet the image!
print("getting Twitter API")
api = get_twitter_api()

# Upload image to Twitter
print(f"uploading {tweet_image_path} to Twitterspace")
media = api.media_upload(tweet_image_path)

# Post tweet with image
print("Tweeting!")
randint = random.randint(1,1000000)
tweet = f"Have you heard about {item}? It's a bullshit product by the bullshit company Nestle. Reminder #{randint} not to purchase this. Thanksx"
post_result = api.update_status(status=tweet, media_ids=[media.media_id])

print("Tweeted!!!")

In [ ]:
def main():
    
if __name__ == "__main__":
    main()

In [ ]:
# TODO: use requests package to download image from search results
# (random Nth result between 1 and 20)